In [24]:
import sqlite3
import csv
import pandas as pd
import numpy as np

In [76]:
# Read data file
df = pd.read_csv('COVID-19 Dataset.csv')

# Convert Date from DD-MM-YYYY to YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y').dt.strftime('%Y-%m-%d')

# Create an SQLite database
conn = sqlite3.connect('covid.db')

# Load data file to SQLite
df.to_sql('covid_pandemic', conn, if_exists='replace')

78386

In [120]:
# Display rows containing null values
query1 = """
SELECT * FROM covid_pandemic 
WHERE "Country/Region" IS NULL 
OR Province IS NULL 
OR Latitude IS NULL 
OR Longitude IS NULL 
OR Date IS NULL 
OR Confirmed IS NULL 
OR Deaths IS NULL 
OR Recovered IS NULL
"""
results1 = pd.read_sql_query(query1, conn)
print(results1)

Empty DataFrame
Columns: [index, Province, Country/Region, Latitude, Longitude, Date, Confirmed, Deaths, Recovered]
Index: []


In [77]:
# Replace null values' rows with 0
query2 = """
UPDATE covid_pandemic
SET "Country/Region" = '0',
    Province = '0',
    Latitude = 0,
    Longitude = 0,
    Date = '0',
    Confirmed = 0,
    Deaths = 0,
    Recovered = 0
WHERE "Country/Region" IS NULL
   OR Province IS NULL
   OR Latitude IS NULL
   OR Longitude IS NULL
   OR Date IS NULL
   OR Confirmed IS NULL
   OR Deaths IS NULL
   OR Recovered IS NULL
"""
conn.execute(query2)
conn.commit()

In [80]:
# Check total number of rows
query3 = """
SELECT COUNT(*) as Total_Rows FROM covid_pandemic
"""
count = pd.read_sql_query(query3, conn)
print(count)

   Total_Rows
0       78386


In [81]:
# start_date and end_date
query4 = """
SELECT MIN(Date) AS start_date, MAX(Date) AS end_date FROM covid_pandemic
"""
Dates = pd.read_sql_query(query4, conn)
print(Dates)

   start_date    end_date
0  2020-01-22  2021-06-13


In [82]:
# Number of months present in the dataset
query5 = """
SELECT COUNT(DISTINCT strftime('%Y-%m', Date)) AS number_of_months FROM covid_pandemic
"""
number_of_months = pd.read_sql_query(query5, conn)
print(number_of_months)

   number_of_months
0                18


In [83]:
# Monthly average for confirmed, deaths, and recovered
query5 = """
SELECT strftime('%Y-%m', Date) AS YearMonth,
AVG(Confirmed) AS average_confirmed,
AVG(Deaths) AS average_deaths,
AVG(Recovered) AS average_recovered
FROM covid_pandemic
GROUP BY YearMonth
ORDER BY YearMonth;
"""
monthly_average = pd.read_sql_query(query5, conn)
print(monthly_average)

   YearMonth  average_confirmed  average_deaths  average_recovered
0    2020-01           4.145455        0.123377           0.092857
1    2020-02          15.296014        0.593596           7.032020
2    2020-03         161.130289        8.660662          27.873900
3    2020-04         505.800433       41.522294         171.642208
4    2020-05         574.849811       30.280897         318.296397
5    2020-06         859.228139       29.817532         548.791558
6    2020-07        1432.361123       35.109552         983.058232
7    2020-08        1611.842899       37.536657        1299.294721
8    2020-09        1784.587446       34.777273        1438.906710
9    2020-10        2412.199623       36.758274        1420.643067
10   2020-11        3592.194372       56.763420        1985.344589
11   2020-12        4050.439673       71.218266        2497.885002
12   2021-01        3911.228530       84.183703        1919.636992
13   2021-02        2433.363636       69.164889        1558.39

In [84]:
# Find most frequent value for confirmed, deaths, recovered each month
query6 = """
WITH MonthlyDeathFrequency AS (
    SELECT
        strftime('%Y-%m', Date) AS Month,
        Deaths,
        COUNT(Deaths) AS FREQ,
        ROW_NUMBER() OVER (PARTITION BY strftime('%Y-%m', Date) ORDER BY COUNT(Deaths) DESC) AS rn
    FROM covid_pandemic
    GROUP BY Month, Deaths
),
MonthlyConfirmedFrequency AS (
    SELECT
        strftime('%Y-%m', Date) AS Month,
        Confirmed,
        COUNT(Confirmed) AS FREQ,
        ROW_NUMBER() OVER (PARTITION BY strftime('%Y-%m', Date) ORDER BY COUNT(Confirmed) DESC) AS rn
    FROM covid_pandemic
    GROUP BY Month, Confirmed
),
MonthlyRecoveredFrequency AS (
    SELECT
        strftime('%Y-%m', Date) AS Month,
        Recovered,
        COUNT(Recovered) AS FREQ,
        ROW_NUMBER() OVER (PARTITION BY strftime('%Y-%m', Date) ORDER BY COUNT(Recovered) DESC) AS rn
    FROM covid_pandemic
    GROUP BY Month, Recovered
)
SELECT 
    df.Month, 
    df.Deaths, 
    df.FREQ AS DeathsFreq, 
    cf.Confirmed, 
    cf.FREQ AS ConfirmedFreq, 
    rf.Recovered, 
    rf.FREQ AS RecoveredFreq
FROM MonthlyDeathFrequency df
JOIN MonthlyConfirmedFrequency cf ON df.Month = cf.Month AND cf.rn = 1
JOIN MonthlyRecoveredFrequency rf ON df.Month = rf.Month AND rf.rn = 1
WHERE df.rn = 1;

"""
frequentpermonth = pd.read_sql_query(query6, conn)
print(frequentpermonth)

      Month  Deaths  DeathsFreq  Confirmed  ConfirmedFreq  Recovered  \
0   2020-01       0        1530          0           1373          0   
1   2020-02       0        4374          0           3926          0   
2   2020-03       0        3815          0           2148          0   
3   2020-04       0        2512          0           1409          0   
4   2020-05       0        2779          0           1607          0   
5   2020-06       0        2745          0           1453          0   
6   2020-07       0        2714          0           1388          0   
7   2020-08       0        2535          0           1260          0   
8   2020-09       0        2349          0           1282          0   
9   2020-10       0        2355          0           1258          0   
10  2020-11       0        2170          0           1190          0   
11  2020-12       0        2180          0           1220          0   
12  2021-01       0        2120          0           1200       

In [85]:
# The minimum values for confirmed, deaths, recovered per year
query7 = """
SELECT strftime('%Y', Date) AS Year, MIN(confirmed) AS min_confirmations, MIN(deaths) AS min_deaths, MIN(recovered) AS min_recoveries
FROM covid_pandemic
GROUP BY Year
"""
min_per_year = pd.read_sql_query(query7, conn)
print(min_per_year)

   Year  min_confirmations  min_deaths  min_recoveries
0  2020                  0           0               0
1  2021                  0           0               0


In [86]:
# The maximum values of confirmed, deaths, recovered per year
query8 = """
SELECT strftime('%Y', Date) AS Year, MAX(confirmed) AS max_confirmations, MAX(deaths) AS max_deaths, MAX(recovered) AS max_recoveries
FROM covid_pandemic
GROUP BY Year
"""
max_per_year = pd.read_sql_query(query8, conn)
print(max_per_year)

   Year  max_confirmations  max_deaths  max_recoveries
0  2020             823225        3752         1123456
1  2021             414188        7374          422436


In [87]:
# The total number of case of confirmed, deaths, recovered each month
query9 = """
SELECT strftime('%Y-%m', Date) AS YearMonth, SUM(Confirmed) AS total_confirmed, SUM(Deaths) AS total_deaths, SUM(Recovered) AS total_recovered
FROM covid_pandemic
GROUP BY YearMonth
"""
number_per_month = pd.read_sql_query(query9, conn)
print(number_per_month)

   YearMonth  total_confirmed  total_deaths  total_recovered
0    2020-01             6384           190              143
1    2020-02            68312          2651            31405
2    2020-03           769236         41346           133070
3    2020-04          2336798        191833           792987
4    2020-05          2744333        144561          1519547
5    2020-06          3969634        137757          2535417
6    2020-07          6838092        167613          4693120
7    2020-08          7694938        179200          6202833
8    2020-09          8244794        160671          6647749
9    2020-10         11515841        175484          6782150
10   2020-11         16595938        262247          9172292
11   2020-12         19336799        339996         11924903
12   2021-01         18672205        401893          9164347
13   2021-02         10492664        298239          6719785
14   2021-03         13924790        282620          7888013
15   2021-04         217

In [46]:
# How corona virus spread out with respect to confirmed cases each month
query10 = """
SELECT strftime('%Y-%m', Date) AS YearMonth, Confirmed
FROM covid_pandemic
"""
confirmed_cases = pd.read_sql_query(query10, conn)
grouped_cases = confirmed_cases.groupby('YearMonth')['Confirmed'].agg(Average = 'mean', Variance = 'var', STD = 'std')
print(grouped_cases)

               Average      Variance           STD
YearMonth                                         
2020-01       4.145455  4.836052e+03     69.541726
2020-02      15.296014  7.850703e+04    280.191051
2020-03     161.130289  1.026629e+06   1013.227134
2020-04     505.800433  7.013581e+06   2648.316704
2020-05     574.849811  6.064851e+06   2462.691765
2020-06     859.228139  1.378219e+07   3712.437842
2020-07    1432.361123  4.692385e+07   6850.098680
2020-08    1611.842899  5.441998e+07   7376.990064
2020-09    1784.587446  6.932971e+07   8326.446123
2020-10    2412.199623  6.900261e+07   8306.781139
2020-11    3592.194372  1.958583e+08  13994.937348
2020-12    4050.439673  4.599818e+08  21447.186252
2021-01    3911.228530  3.163710e+08  17786.819944
2021-02    2433.363636  7.960638e+07   8922.240920
2021-03    2916.797235  8.374281e+07   9151.109600
2021-04    4699.355195  5.011217e+08  22385.747124
2021-05    4005.254085  6.287793e+08  25075.472447
2021-06    2508.632368  1.10988

In [49]:
# How corona virus spread out with respect to death cases each month
query11 = """
SELECT strftime('%Y-%m', Date) AS YearMonth, Deaths
FROM covid_pandemic
"""
death_cases = pd.read_sql_query(query11, conn)
grouped_cases = death_cases.groupby('YearMonth')['Deaths'].agg(Average = 'mean', Variance = 'var', STD = 'std')
print(grouped_cases)

             Average       Variance         STD
YearMonth                                      
2020-01     0.123377       4.248576    2.061207
2020-02     0.593596      68.337150    8.266629
2020-03     8.660662    3901.609527   62.462865
2020-04    41.522294   40513.037173  201.278506
2020-05    30.280897   20689.245405  143.837566
2020-06    29.817532   16933.110885  130.127287
2020-07    35.109552   21144.584057  145.411774
2020-08    37.536657   23277.872425  152.570877
2020-09    34.777273   20107.121415  141.799582
2020-10    36.758274   17583.754253  132.603749
2020-11    56.763420   27779.806542  166.672753
2020-12    71.218266   65359.059830  255.654180
2021-01    84.183703  102779.961427  320.593140
2021-02    69.164889   68494.756150  261.715029
2021-03    59.199832   54397.364207  233.232425
2021-04    78.438745   94631.954030  307.623071
2021-05    76.780268  131797.076577  363.038671
2021-06    66.262238  113020.126599  336.184661


In [51]:
# How corona virus spread out with respect to recovered cases each month
query12 = """
SELECT strftime('%Y-%m', Date) AS YearMonth, Recovered
FROM covid_pandemic
"""
recovered_cases = pd.read_sql_query(query12, conn)
grouped_cases = recovered_cases.groupby('YearMonth')['Recovered'].agg(Average = 'mean', Variance = 'var', STD = 'std')
print(grouped_cases)

               Average      Variance           STD
YearMonth                                         
2020-01       0.092857  2.635297e+00      1.623360
2020-02       7.032020  1.244945e+04    111.577102
2020-03      27.873900  4.012159e+04    200.303754
2020-04     171.642208  7.700597e+05    877.530462
2020-05     318.296397  1.978621e+06   1406.634592
2020-06     548.791558  6.531586e+06   2555.696824
2020-07     983.058232  2.484908e+07   4984.885449
2020-08    1299.294721  4.017884e+07   6338.677968
2020-09    1438.906710  5.703591e+07   7552.212383
2020-10    1420.643067  7.374715e+07   8587.616093
2020-11    1985.344589  5.073860e+07   7123.103344
2020-12    2497.885002  3.267632e+08  18076.591784
2021-01    1919.636992  3.150030e+07   5612.512665
2021-02    1558.391698  2.443308e+07   4942.982693
2021-03    1652.285924  3.490470e+07   5908.020232
2021-04    3074.785065  2.244682e+08  14982.261890
2021-05    4007.507750  7.553337e+08  27483.335860
2021-06    2769.449550  2.33150

In [72]:
# The country having highest number of the confirmed cases
query13 = """
SELECT "Country/Region", SUM(Confirmed) AS Confirmed_Cases
FROM covid_pandemic
GROUP BY "Country/Region"
ORDER BY Confirmed_Cases DESC LIMIT 1
"""
result = pd.read_sql_query(query13, conn)
print(result)

  Country/Region  Confirmed_Cases
0             US         33461982


In [73]:
# The country having lowest number of the death cases
query14 = """
SELECT "Country/Region", SUM(Deaths) AS Death_Cases
FROM covid_pandemic
GROUP BY "Country/Region"
ORDER BY Death_Cases LIMIT 1
"""
result = pd.read_sql_query(query14, conn)
print(result)

  Country/Region  Death_Cases
0       Dominica            0


In [74]:
# Top 5 countries having highest recovered case
query15 = """
SELECT "Country/Region", SUM(Recovered) AS Recovered_Cases
FROM covid_pandemic
GROUP BY "Country/Region"
ORDER BY Recovered_Cases DESC LIMIT 5
"""
top_five = pd.read_sql_query(query15, conn)
print(top_five)

  Country/Region  Recovered_Cases
0          India         28089649
1         Brazil         15400169
2             US          6303715
3         Turkey          5202251
4         Russia          4745756
